In [1]:
%load_ext sql

In [2]:
%sql sqlite://

### Задание 1

 Посчитать % изменение количества клиентов, совершивших покупку, месяц-к-месяцу.

In [3]:
%%sql
CREATE TABLE Orders(
        client_id INTEGER,
        order_id INTEGER,
        order_date TEXT);

INSERT INTO Orders
VALUES (500, 11, DATETIME('2021-01-01')),
       (507, 77, DATETIME('2021-01-02')),
       (53, 50, DATETIME('2021-08-01')),
       (105, 48, DATETIME('2021-11-11')),
       (106, 48, DATETIME('2021-11-19')),
       (107, 48, DATETIME('2021-11-13')),
       (105, 48, DATETIME('2021-11-11')),
       (50, 10, DATETIME('2021-11-05')),
       (53, 12, DATETIME('2021-08-12'));
        
SELECT *
  FROM Orders

 * sqlite://
Done.
9 rows affected.
Done.


client_id,order_id,order_date
500,11,2021-01-01 00:00:00
507,77,2021-01-02 00:00:00
53,50,2021-08-01 00:00:00
105,48,2021-11-11 00:00:00
106,48,2021-11-19 00:00:00
107,48,2021-11-13 00:00:00
105,48,2021-11-11 00:00:00
50,10,2021-11-05 00:00:00
53,12,2021-08-12 00:00:00


In [14]:
%%sql
WITH orders_by_month AS(
            SELECT COUNT(DISTINCT client_id) AS qty,
                   strftime('%m', order_date) AS Month,
                   LAG(COUNT(DISTINCT client_id), 1, 0) OVER(ORDER BY 2) AS prev_qty
              FROM Orders
             GROUP BY 2
                        )

SELECT Month,
       ((qty - prev_qty) * 100 / 
                        prev_qty) || "%" AS diff_pct
  FROM orders_by_month;

 * sqlite://
Done.


Month,diff_pct
01,None
08,-50%
11,300%


### Задание 2
Вывести сумму GMV (Gross Merchandise Value) с нарастающим итогом по дням

In [6]:
%%sql
CREATE TABLE Gross(
        fact_date TEXT,
        gmv INTEGER);

INSERT INTO Gross
VALUES (DATETIME('2021-01-01'), 4372888),
       (DATETIME('2021-11-01'), 1111111),
       (DATETIME('2021-07-07'), 3245346),
       (DATETIME('2021-11-11'), 123445),
       (DATETIME('2021-07-01'), 77),
       (DATETIME('2021-05-01'), 45430);
    
SELECT *
  FROM Gross;

 * sqlite://
Done.
6 rows affected.
Done.


fact_date,gmv
2021-01-01 00:00:00,4372888
2021-11-01 00:00:00,1111111
2021-07-07 00:00:00,3245346
2021-11-11 00:00:00,123445
2021-07-01 00:00:00,77
2021-05-01 00:00:00,45430


In [13]:
%%sql
SELECT fact_date,
       SUM(gmv) OVER(ORDER BY fact_date) AS run_total
  FROM Gross
 ORDER BY 1;

 * sqlite://
Done.


fact_date,run_total
2021-01-01 00:00:00,4372888
2021-05-01 00:00:00,4418318
2021-07-01 00:00:00,4418395
2021-07-07 00:00:00,7663741
2021-11-01 00:00:00,8774852
2021-11-11 00:00:00,8898297


### Задание 3

Получить время отклика на каждое письмо (письмо идентифицируется по полю mail_id), отправленное пользователем mr_employee@ozon.ru. 

Дана таблица с логом электронных писем пользователя mr_employee@ozon.ru (т.е. письма, отправленные с этой электронной почты и полученные на нее). У всех цепочек сообщений уникальная тема. В одной цепочке может быть несколько писем.

In [8]:
%%sql
CREATE TABLE  mails(
        mail_id INTEGER,
        mail_from TEXT,
        mail_to TEXT,
        mail_subject TEXT,
        timestamp TEXT);

INSERT INTO mails
VALUES (1, 'mr_employee@ozon.ru', 'ms_intern@ozon.ru', 'Test', DATETIME('2021-01-08 12:00:03')),
       (2, 'ms_intern@ozon.ru', 'mr_employee@ozon.ru', 'Test', DATETIME('2021-01-10 13:41:34')),
       (3, 'mr_employee@ozon.ru', 'mr_boss@ozon.ru', 'Q2', DATETIME('2021-01-11 15:02:57')),
       (4, 'mr_boss@ozon.ru', 'mr_employee@ozon.ru', 'Q2', DATETIME('2021-01-18 11:03:08')),
       (5, 'mr_employee@ozon.ru', 'ms_intern@ozon.ru', 'Test', DATETIME('2021-01-20 19:48:54')),
       (6, 'ms_intern@ozon.ru', 'mr_employee@ozon.ru', 'Test', DATETIME('2021-01-21 09:48:54')),
       (7, 'mr_employee@ozon.ru', 'ms_intern@ozon.ru', 'Test', DATETIME('2021-01-21 17:48:54'));
        
SELECT *
  FROM mails
 ORDER BY 5;

 * sqlite://
Done.
7 rows affected.
Done.


mail_id,mail_from,mail_to,mail_subject,timestamp
1,mr_employee@ozon.ru,ms_intern@ozon.ru,Test,2021-01-08 12:00:03
2,ms_intern@ozon.ru,mr_employee@ozon.ru,Test,2021-01-10 13:41:34
3,mr_employee@ozon.ru,mr_boss@ozon.ru,Q2,2021-01-11 15:02:57
4,mr_boss@ozon.ru,mr_employee@ozon.ru,Q2,2021-01-18 11:03:08
5,mr_employee@ozon.ru,ms_intern@ozon.ru,Test,2021-01-20 19:48:54
6,ms_intern@ozon.ru,mr_employee@ozon.ru,Test,2021-01-21 09:48:54
7,mr_employee@ozon.ru,ms_intern@ozon.ru,Test,2021-01-21 17:48:54


In [9]:
%%sql
WITH prev_mail AS (
                SELECT mail_id,
                       LAG(timestamp, 1, 0) OVER(PARTITION BY mail_subject
                                ORDER BY timestamp) AS prev_timestamp
                  FROM mails
                  )

SELECT m.mail_id,
       CAST((JULIANDAY(m.timestamp) - JULIANDAY(pm.prev_timestamp)) * 24 * 60
                   AS INTEGER) AS respond_minutes
  FROM mails AS m
       JOIN prev_mail AS pm
       ON pm.mail_id = m.mail_id
 WHERE prev_timestamp != 0
   AND m.mail_to = 'mr_employee@ozon.ru';

 * sqlite://
Done.


mail_id,respond_minutes
2,2981
4,9840
6,840


### Задание 4

Вывести id сотрудников с разницей в заработной плате в пределах 5000 рублей.

Вывожу любой id, если есть хоть ещё один сотрудник с зп +-5000.

In [10]:
%%sql
CREATE TABLE salaries(
        employee_id INTEGER,
        salary_rub INTEGER);

INSERT INTO salaries
VALUES (1, 11111),
       (2, 111110),
       (3, 13111),
       (4, 111),
       (5, 11111),
       (6, 51111),
       (7, 55111),
       (8, 11110),
       (9, 1131100);
        
SELECT *
  FROM salaries;

 * sqlite://
Done.
9 rows affected.
Done.


employee_id,salary_rub
1,11111
2,111110
3,13111
4,111
5,11111
6,51111
7,55111
8,11110
9,1131100


In [11]:
%%sql
SELECT DISTINCT s1.employee_id
  FROM salaries AS s1, salaries AS s2
 WHERE s2.salary_rub BETWEEN (s1.salary_rub - 5000)
                         AND (s1.salary_rub + 5000)
   AND s1.employee_id != s2.employee_id;

 * sqlite://
Done.


employee_id
1
3
5
6
7
8
